# 🎯 05번: Metadata Filtering

## 🎯 학습 목표
- **메타데이터 필터**로 정확한 문서 검색
- **동음이의어** 문제 해결 (파스텔 = 호텔? 카페? 극장?)

## 💡 핵심 아이디어

벡터 검색만으로는 "파스텔 예약"이 호텔인지 카페인지 구분 못함!

→ **메타데이터 필터**로 해결:
```python
vectorstore.similarity_search(
    "파스텔 예약", 
    filter={"type": "hotel"}  # 호텔만!
)
```

## 📋 실습 구성
1. 동음이의어 문제 확인
2. 메타데이터 필터링 기본
3. 퀴즈 1: 상황별 필터 선택
4. 퀴즈 2: 틀린 필터 찾기

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas matplotlib

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
import matplotlib.pyplot as plt

print("✅ 라이브러리 준비 완료!")

In [ ]:
# 01-03번과 동일한 문서 (메타데이터 추가)
pastel_docs = [
    {
        "content": """파스텔 호텔 예약 안내
서울 강남구에 위치한 파스텔 호텔입니다.
- 예약 변경: 투숙 3일 전까지 가능 (위약금 없음)
- 주말 요금: 평일 대비 30% 할인""",
        "type": "hotel",
        "location": "강남",
        "features": ["예약변경", "주말할인"]
    },
    {
        "content": """파스텔 카페 메뉴
홍대 파스텔 카페는 디저트로 유명합니다.
- 예약: 단체 예약만 가능 (10명 이상)
- 주말: 마카롱 만들기 체험 (14:00-16:00)""",
        "type": "cafe",
        "location": "홍대",
        "features": ["단체예약", "체험수업"]
    },
    {
        "content": """파스텔 소극장 공연 안내
대학로 파스텔 소극장에서 뮤지컬을 상영합니다.
- 티켓 예약: 인터파크, 현장 구매 가능
- 주말: 토요일 15:00 추가 공연""",
        "type": "theater",
        "location": "대학로",
        "features": ["뮤지컬", "주말공연"]
    },
    {
        "content": """국립현대미술관 전시 정보
서울 국립현대미술관 덕수궁관 전시 안내입니다.
- 주말 관람 팁: 토요일 14:00-16:00 혼잡, 일요일 오전 10:00-12:00 한적함""",
        "type": "museum",
        "location": "덕수궁",
        "features": ["전시", "주말정보"]
    },
    {
        "content": """서울 여행 블로그
미술관은 주말 오전이 베스트 타이밍.
예약은 미리미리! 특히 숙소는 필수.""",
        "type": "blog",
        "location": "서울",
        "features": ["여행정보"]
    },
    {
        "content": """서울역 교통 안내
KTX, 지하철 1/4호선 환승 가능.
주말 막차 시간: 자정까지 운행""",
        "type": "transport",
        "location": "서울역",
        "features": ["교통", "환승"]
    },
    {
        "content": """주말 날씨 예보
이번 주말 서울 날씨는 맑음.
외출하기 좋은 날씨입니다.""",
        "type": "weather",
        "location": "서울",
        "features": ["날씨"]
    },
    {
        "content": """예약 플랫폼 가이드
네이버, 카카오 예약 시스템 비교.
주말 예약은 평일보다 2배 빠르게 마감됩니다.""",
        "type": "guide",
        "location": "온라인",
        "features": ["예약정보"]
    }
]

# Document 객체 생성
documents = [Document(
    page_content=doc["content"], 
    metadata={"type": doc["type"], "location": doc["location"]}
) for doc in pastel_docs]

# 벡터스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ 벡터스토어 준비 완료!")
print(f"   문서 수: {len(documents)}개")
print("\n📋 메타데이터 예시:")
for i, doc in enumerate(pastel_docs[:3], 1):
    print(f"   {i}. {doc['type']} ({doc['location']}): {doc['content'].split(chr(10))[0]}")

## 🔴 Step 1: 동음이의어 문제 재현

### 문제 상황
사용자가 **"파스텔 예약"**이라고 질문하면?

- 🏨 파스텔 **호텔** 예약?
- ☕ 파스텔 **카페** 예약?
- 🎭 파스텔 **소극장** 예약?

### 벡터 검색의 한계
"예약"이라는 키워드에 모두 반응해서 **섞여서 나옴**!

지금부터 **실제로 확인**해봅시다.

In [ ]:
# 동음이의어 테스트
query = "파스텔 예약 변경하고 싶어요"

print("❓ 질문:")
print(f"   {query}")
print()

# 필터 없이 검색
results = vectorstore.similarity_search(query, k=3)

print("🔍 검색 결과 (Top-3):")
for i, doc in enumerate(results, 1):
    doc_type = doc.metadata['type']
    location = doc.metadata['location']
    first_line = doc.page_content.split('\n')[0]
    print(f"   {i}. [{doc_type}] {location} - {first_line}")
print()

print("❌ 문제:")
print("   - hotel, cafe, theater가 섞여서 나옴!")
print("   - 사용자가 원한 건 '호텔' 예약인데...")
print("   - 벡터 검색만으로는 의도 파악 불가능!")

## 💡 Step 2: Metadata Filtering 개념

### 📌 언제 사용하나요?

동일한 키워드가 여러 의미를 가질 때 메타데이터로 구분할 수 있습니다:

**✅ 사용하기 좋은 경우:**
- "파스텔 예약" ← 호텔? 카페? 극장? (동음이의어)
- "서울 맛집" ← 지역별 필터링 필요
- "2024년 1월 뉴스" ← 날짜 필터링 필요
- "IT 기술 문서" ← 카테고리 필터링 필요

**❌ 효과가 적은 경우:**
- 문서 종류가 1개만 있을 때
- 모든 문서가 같은 카테고리일 때
- 필터링할 메타데이터가 없을 때

---

### 메타데이터란?
**"데이터를 설명하는 데이터"** - 문서의 속성 정보

```python
{
    "content": "파스텔 호텔 예약 안내...",
    "metadata": {
        "type": "hotel",        # 문서 종류
        "location": "강남"      # 위치 정보
    }
}
```

### 필터링 원리

**검색 전략:**
```
1. 벡터 검색: "예약" 관련 문서 찾기
2. 메타데이터 필터: type="hotel"만 남기기
3. 결과: 호텔 예약 정보만 정확히 추출!
```

### 실습: 직접 필터를 만들어봅시다!

아래 셀에서 **여러분이 직접** 필터를 작성해보세요.

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 호텔만 검색하고 싶을 때 어떤 필터를 작성해야 할까요?

my_filter = {}  # 여기에 필터를 작성하세요

print("📝 내가 만든 필터:")
print(f"   {my_filter}")
print()
print("💡 힌트: 메타데이터의 'type' 키를 사용하세요")
print("💡 아래 퀴즈 셀에서 실제로 테스트해볼 수 있습니다!")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
# 호텔만 검색하고 싶을 때
my_filter = {"type": "hotel"}
```

**다른 필터 예시:**
```python
# 카페만 검색
my_filter = {"type": "cafe"}

# 강남 지역만 검색
my_filter = {"location": "강남"}

# 호텔이면서 강남인 것만
my_filter = {"type": "hotel", "location": "강남"}
```

</details>

In [ ]:
# 필터링 검색 함수
def search_with_filter(query, metadata_filter=None, k=3):
    """메타데이터 필터를 적용한 검색"""
    # 1. 많이 검색 (fetch_k=30)
    all_results = vectorstore.similarity_search(query, k=30)
    
    # 2. 필터 적용
    if metadata_filter:
        filtered = []
        for doc in all_results:
            # 모든 필터 조건 체크
            match = all(
                doc.metadata.get(key) == value 
                for key, value in metadata_filter.items()
            )
            if match:
                filtered.append(doc)
    else:
        filtered = all_results
    
    # 3. Top-K 반환
    return filtered[:k]

print("✅ 필터링 검색 함수 준비 완료!")

---

### 🎯 퀴즈 1: 상황별 필터 선택하기

**상황**: "강남에 있는 호텔만 검색하고 싶어요"

어떤 필터를 사용해야 할까요?

**A)** `{"type": "hotel"}`  
**B)** `{"location": "강남"}`  
**C)** `{"type": "hotel", "location": "강남"}`

아래 셀에서 선택해보세요!

In [ ]:
# 여기를 수정하세요! 👇
# A, B, C 중 하나를 선택하세요

# A) type만
# my_answer = {"type": "hotel"}

# B) location만
# my_answer = {"location": "강남"}

# C) 둘 다
my_answer = {}  # 여기에 답을 작성하세요

print("📝 내가 선택한 필터:")
print(f"   {my_answer}")
print()

# 테스트
query = "강남 호텔 예약"
results = search_with_filter(query, metadata_filter=my_answer, k=5)

print(f"🔍 검색 결과:")
for i, doc in enumerate(results, 1):
    doc_type = doc.metadata['type']
    location = doc.metadata['location']
    print(f"   {i}. [{doc_type}] {location}")

# 정답 확인
hotel_and_gangnam = [doc for doc in results if doc.metadata['type'] == 'hotel' and doc.metadata['location'] == '강남']
if hotel_and_gangnam and my_answer == {"type": "hotel", "location": "강남"}:
    print("\n✅ 정답! 강남에 있는 호텔만 검색되었습니다!")
elif my_answer == {"type": "hotel"}:
    print("\n△ 부분 정답! 호텔만 나왔지만 강남이 아닌 곳도 섞였어요.")
elif my_answer == {"location": "강남"}:
    print("\n△ 부분 정답! 강남만 나왔지만 카페/극장도 섞였어요.")
else:
    print("\n❌ 다시 시도해보세요!")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: C)** `{"type": "hotel", "location": "강남"}`

**해설:**
- A) `{"type": "hotel"}` → 호텔만 나오지만 강남/홍대/대학로 모두 섞임
- B) `{"location": "강남"}` → 강남만 나오지만 호텔/카페 모두 섞임
- **C) 둘 다** → 강남에 있는 호텔만 정확히 검색! ✅

**핵심:**
여러 조건을 동시에 만족시키려면 필터에 모두 넣으면 됩니다!

</details>

---

### 🎯 퀴즈 2: 틀린 필터 찾기

아래 3개 필터 중 **작동하지 않는 것**은?

**A)** `{"type": "hotel"}`  
**B)** `{"type": "Hotel"}`  ← 대문자  
**C)** `{"category": "hotel"}`  ← 키 이름 다름

아래 셀에서 각각 테스트해보세요!

In [ ]:
# 3가지 필터 테스트
filters_to_test = [
    {"type": "hotel"},        # A
    {"type": "Hotel"},        # B
    {"category": "hotel"},    # C
]

query = "파스텔 호텔 예약"

print("🔍 필터 테스트")
print("=" * 60)
print()

for i, filter_test in enumerate(filters_to_test, 1):
    label = chr(64 + i)  # A, B, C
    print(f"{label}) {filter_test}")
    
    results = search_with_filter(query, metadata_filter=filter_test, k=3)
    
    if results:
        print(f"   검색됨: {len(results)}개")
        for doc in results[:2]:
            print(f"   - [{doc.metadata['type']}] {doc.metadata['location']}")
    else:
        print(f"   ❌ 검색 결과 없음!")
    print()

print("💡 힌트: 메타데이터의 키와 값은 대소문자까지 정확히 일치해야 합니다!")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: B와 C 모두 작동 안 함!**

**A)** `{"type": "hotel"}` → ✅ 작동 (정확한 키와 값)

**B)** `{"type": "Hotel"}` → ❌ 작동 안 함  
- 메타데이터는 `"hotel"` (소문자)인데 필터는 `"Hotel"` (대문자)
- 대소문자까지 정확히 일치해야 함!

**C)** `{"category": "hotel"}` → ❌ 작동 안 함  
- 메타데이터 키는 `"type"`인데 필터는 `"category"`
- 키 이름이 틀림!

**핵심:**
메타데이터 필터는 **키 이름과 값 모두 정확히 일치**해야 합니다!
- 대소문자 구분: `"hotel"` ≠ `"Hotel"`
- 키 이름 확인: `"type"` ≠ `"category"`

</details>

## 🔍 Step 3: 필터링 적용 및 비교

### FAISS 필터링 방법
FAISS는 **사후 필터링** 방식:
1. 먼저 많은 문서 검색 (k=30)
2. 메타데이터 조건으로 필터링
3. 최종 결과 반환 (top-3)

### 구현 전략
```python
# 1. 많이 검색
docs = vectorstore.similarity_search(query, k=30)

# 2. 필터 적용
filtered = [doc for doc in docs if doc.metadata['type'] == 'hotel']

# 3. Top-K 반환
result = filtered[:3]
```

In [ ]:
# 필터 전/후 비교
query = "파스텔 예약 변경하고 싶어요"

print("🔍 필터링 전후 비교")
print("=" * 60)
print(f"\n❓ 질문: {query}\n")

# 필터 없음
print("❌ 필터 없음:")
no_filter = search_with_filter(query, metadata_filter=None, k=3)
for i, doc in enumerate(no_filter, 1):
    doc_type = doc.metadata['type']
    print(f"   {i}. {doc_type}")
print()

# 필터 적용
print("✅ 필터 적용 (type=hotel):")
with_filter = search_with_filter(query, metadata_filter={"type": "hotel"}, k=3)
for i, doc in enumerate(with_filter, 1):
    doc_type = doc.metadata['type']
    location = doc.metadata['location']
    first_line = doc.page_content.split('\n')[0]
    print(f"   {i}. [{doc_type}] {location}")
    print(f"      {first_line}")
print()

print("💡 결과:")
print("   - 필터 없음: hotel, cafe, theater 섞임 ❌")
print("   - 필터 적용: hotel만 정확히 검색! ✅")

## 🎓 핵심 정리

### Metadata Filtering이란?

**문제:** 벡터 검색만으로는 동음이의어 구분 불가  
→ "파스텔 예약" = 호텔? 카페? 극장?

**해결:** 메타데이터로 필터링!
```python
filter = {"type": "hotel"}  # 호텔만 검색
filter = {"type": "hotel", "location": "강남"}  # 강남 호텔만
```

### 언제 사용?
- ✅ 동음이의어가 많을 때
- ✅ 문서 종류/카테고리가 다양할 때
- ✅ 지역/날짜 등으로 필터링 필요할 때

### 주의사항
- 키와 값이 **정확히 일치**해야 함
- 대소문자 구분: `"hotel"` ≠ `"Hotel"`
- 키 이름 확인: `"type"` ≠ `"category"`

---

**🚀 다음 단계:** 06번 Hybrid Search + Re-ranking

In [ ]:
# 🎉 05번 완료!

print("✅ 배운 내용:")
print("   - 메타데이터 필터로 정확한 검색")
print("   - 동음이의어 문제 해결")
print("   - filter={'type': 'hotel'}")
print()
print("🚀 다음: 06번 Hybrid Search + Re-ranking")